In [1]:
import os
import numpy as np
import pandas as pd

In [2]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [3]:
project_path = "/content/drive/MyDrive/Colab/SRL/"

In [4]:
!pip install conllu

In [33]:
from conllu import parse
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import precision_recall_fscore_support

In [32]:
from sklearn.svm import LinearSVC
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline

In [26]:
train_path = project_path + "train.conllu"
with open(train_path, "r", encoding="utf-8") as file:
    data = file.read()
# sentences = parse(data)

In [27]:
# for i in range(0, 5):
#     print(sentences[i])
# print(len(sentences))

In [8]:
trainfile = project_path + "train.csv"

In [74]:
# df['postposition']
# df['chunk']
# df['dependency']
# df['dependency-head']
df = pd.read_csv(trainfile, sep=',')
df.drop(columns=['dependency-head', 'chunk'], inplace=True)

In [90]:
# A = df['chunk'].tolist()
B = df['dependency'].tolist()
pos_tags = set(df['head-POS'].tolist())
# print(pos_tags)
print(len(df))
srlabels = set(df['srl'].tolist())
print(srlabels)
# print(df)
print(df.columns)
print(df.head())

14546
{nan, 'ARG2-GOL', 'ARGM-CAU', 'ARGM-DIR', 'ARG2-LOC', 'ARGM-MOD', 'ARG2', 'ARGM-LOC', 'ARGM-MNR', 'ARGM-MNS', 'ARG0', 'ARGM-TMP', 'ARGM-DIS', 'ARGM-ADV', 'ARG1', 'ARG3', 'ARGM-PRP', 'ARGM-NEG', 'ARG2-ATR', 'ARG2-SOU', 'ARGM-EXT', 'ARG-UNDEF', 'ARGM-PRX'}
Index(['word', 'postposition', 'dependency', 'is_arg', 'srl', 'predicate',
       'head-POS'],
      dtype='object')
        word postposition dependency  is_arg       srl predicate head-POS
0  उल्लेखनीय            0        k1s       1  ARG2-ATR     VGF.1      adj
1         है           है       root       0       NaN       NaN        v
2         कि            0         rs       0       NaN       NaN      avy
3    अक्तूबर         0_को        k7t       1  ARGM-TMP      VGNF        n
4      NULL2            0       root       0       NaN       NaN      NaN


In [96]:
inputs1 = df[['word','is_arg', 'predicate', 'postposition']]
inputs2 = df[['word', 'postposition', 'is_arg', 'predicate', 'head-POS']]
inputs3 = df[['word','dependency', 'postposition', 'is_arg', 'predicate', 'head-POS']]
targets = df['srl']
# print(df['word'].tolist())
# print(set(df['dependency'].tolist()))
print(inputs1.head())

        word  is_arg predicate postposition
0  उल्लेखनीय       1     VGF.1            0
1         है       0       NaN           है
2         कि       0       NaN            0
3    अक्तूबर       1      VGNF         0_को
4      NULL2       0       NaN            0


In [97]:
def preprocess(inputs, targets):
    X = inputs.copy()
    label_encoder = LabelEncoder()
    label_encoders = {}

    for col in X.columns:
        if X[col].dtype == 'object':
            label_encoders[col] = LabelEncoder()
            X[col] = label_encoders[col].fit_transform(X[col])
    Y = label_encoder.fit_transform(targets)
    Xtrain, Xtest, Ytrain, Ytest = train_test_split(X, Y, test_size=0.1, random_state=42)
    return Xtrain, Xtest, Ytrain, Ytest

In [98]:
Xtrain1, Xtest1, Ytrain1, Ytest1 = preprocess(inputs1, targets)
Xtrain2, Xtest2, Ytrain2, Ytest2 = preprocess(inputs2, targets)
Xtrain3, Xtest3, Ytrain3, Ytest3 = preprocess(inputs3, targets)

In [99]:
def train(Xtrain, Ytrain):
    svm = make_pipeline(StandardScaler(), LinearSVC())
    svm.fit(Xtrain, Ytrain)
    return svm

In [100]:
model1 = train(Xtrain1, Ytrain1)
model2 = train(Xtrain2, Ytrain2)
model3 = train(Xtrain3, Ytrain3)

In [101]:
def getMetrics(model, Xtest, Ytest):
    Ypred = model.predict(Xtest)
    a = accuracy_score(Ytest, Ypred)
    p, r, f1, _ = precision_recall_fscore_support(Ytest, Ypred, average='weighted')
    return Ypred, a, p, r, f1

In [102]:
_, a1, p1, r1, f1 = getMetrics(model1, Xtest1, Ytest1)
_, a2, p2, r2, f2 = getMetrics(model2, Xtest2, Ytest2)
Ypred, a3, p3, r3, f3 = getMetrics(model3, Xtest3, Ytest3)

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [103]:
print(f"\nAccuracy: {a1 * 100.:4f} %")
print(f"\nPrecision: {p1 * 100.:4f} ")
print(f"\nRecall: {r1 * 100.:4f} ")
print(f"\nF1 score: {f1 * 100.:4f} ")


Accuracy: 76.941581 %

Precision: 69.274082 

Recall: 76.941581 

F1 score: 71.948644 


In [104]:
print(f"\nAccuracy: {a2 * 100.:4f} %")
print(f"\nPrecision: {p2 * 100.:4f} ")
print(f"\nRecall: {r2 * 100.:4f} ")
print(f"\nF1 score: {f2 * 100.:4f} ")


Accuracy: 77.216495 %

Precision: 73.405534 

Recall: 77.216495 

F1 score: 72.749862 


In [105]:
print(f"\nAccuracy: {a3 * 100.:4f} %")
print(f"\nPrecision: {p3 * 100.:4f} ")
print(f"\nRecall: {r3 * 100.:4f} ")
print(f"\nF1 score: {f3 * 100.:4f} ")


Accuracy: 81.271478 %

Precision: 77.528466 

Recall: 81.271478 

F1 score: 78.004772 


In [61]:
# predicted_labels = label_encoder.inverse_transform(Ypred)
# print(Xtest.columns)
# actual_words = label_encoders['word'].inverse_transform(Xtest['word'])
# print(actual_words[100:110])
# print(predicted_labels[100:110])

['ग्रह' 'पार्टी' 'कदम' 'नियमों' 'भारत' 'गोले' 'उन्होंने' 'डांस' 'भूमिका'
 'करने']
[nan 'ARG1' nan 'ARG1' nan 'ARG1' 'ARG1' 'ARG1' nan 'ARGM-MNR']
